In [1]:
import os
import sys
os.environ['JAVA_HOME'] = "/home/hbg/spark/jdk-11.0.2"
sys.path.append('/home/hbg/spark/spark-3.3.2-bin-hadoop3/python/')
sys.path.append('/home/hbg/spark/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip')

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/02/24 06:01:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/24 06:01:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/24 06:01:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
spark.version

'3.3.2'

In [5]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [6]:
df = df.repartition(4)

In [7]:
df.write.parquet('yellow/2024/10/', mode='overwrite')

In [8]:
!ls -lh yellow/2024/10/

total 97M
-rw-r--r-- 1 hbg hbg   0 Feb 24 06:03 _SUCCESS
-rw-r--r-- 1 hbg hbg 25M Feb 24 06:03 part-00000-96719601-3ad4-4f92-a6b4-d6dde6ad6bf2-c000.snappy.parquet
-rw-r--r-- 1 hbg hbg 25M Feb 24 06:03 part-00001-96719601-3ad4-4f92-a6b4-d6dde6ad6bf2-c000.snappy.parquet
-rw-r--r-- 1 hbg hbg 25M Feb 24 06:03 part-00002-96719601-3ad4-4f92-a6b4-d6dde6ad6bf2-c000.snappy.parquet
-rw-r--r-- 1 hbg hbg 25M Feb 24 06:03 part-00003-96719601-3ad4-4f92-a6b4-d6dde6ad6bf2-c000.snappy.parquet


In [9]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-10-07 16:40:43|  2024-10-07 18:10:56|              1|         14.8|        99|                 N|         127|         225|           1|       47.5|  0.0|    0.5|       0.

In [11]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [18]:
df.createOrReplaceTempView('yellow_2024_10')

In [19]:
spark.sql("""
SELECT
    COUNT(1)
FROM 
    yellow_2024_10
WHERE
    to_date(tpep_pickup_datetime) = '2024-10-15';
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [30]:
spark.sql("""
SELECT
    MAX((CAST(tpep_dropoff_datetime AS LONG) - CAST(tpep_pickup_datetime AS LONG)) / 3600) AS duration
FROM 
    yellow_2024_10;
""").show()

+------------------+
|          duration|
+------------------+
|162.61777777777777|
+------------------+



In [31]:
zones_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [32]:
df_zones = spark.read.option("header", "true").schema(zones_schema).csv('taxi_zone_lookup.csv')

In [33]:
df_zones.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [34]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [35]:
df_zones.createOrReplaceTempView('zones')

In [39]:
spark.sql("""
SELECT
    zones.Zone AS pickup_location_zone,
    COUNT(1)
FROM 
    yellow_2024_10 yellow LEFT JOIN zones ON yellow.PULocationID = zones.LocationID
GROUP BY 
    1
ORDER BY
    2 ASC;
""").show(10, False)

+---------------------------------------------+--------+
|pickup_location_zone                         |count(1)|
+---------------------------------------------+--------+
|Governor's Island/Ellis Island/Liberty Island|1       |
|Rikers Island                                |2       |
|Arden Heights                                |2       |
|Jamaica Bay                                  |3       |
|Green-Wood Cemetery                          |3       |
|West Brighton                                |4       |
|Rossville/Woodrow                            |4       |
|Charleston/Tottenville                       |4       |
|Port Richmond                                |4       |
|Eltingville/Annadale/Prince's Bay            |4       |
+---------------------------------------------+--------+
only showing top 10 rows

